In [9]:
import statsmodels.api as sm
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt

In [10]:
N = 50
sample = stats.uniform.rvs(size=N*5)*2 - 1
sample = sample.reshape((N, 5))
eta = np.zeros((N))

C = [2, 3, -2, 1, 1, -1]

for i in range(N):
    eta[i] = np.random.normal(C[0] + C[1:] @ sample[i].T, 1.5)

In [11]:
def rss0(x):
    return sum([(a - np.mean(x))**2 for a in x])


In [12]:
def regression(psi, eta):

    F = psi.T @ psi
    F_inv = np.linalg.inv(F)
    b = (F_inv @ psi.T) @ eta.T
    e = eta - (psi @ b.T).T
    RSSy = e @ e.T
    RSS0y = rss0(eta)

    R = (RSS0y - RSSy)/RSS0y
    
    delta = [b[i]*((N - len(psi[0]))/(RSSy*F_inv[i][i]))**(1/2) \
             for i in range(len(psi[0]))]
    b_p_value = 2*(1 - stats.t.cdf(np.abs(delta), N - np.shape(psi)[1]))
    
    return b, b_p_value, RSSy, RSS0y, R

In [13]:
RSS0_xi = np.zeros(5)
RSS_xi = np.zeros(5)
R_xi = np.zeros(5)

for i in range(5):
    psi_xi = np.copy(sample)
    for j in range(N):
        psi_xi[j][i] = 1
    
    RSS_xi[i] = regression(psi_xi, sample.T[i])[2]
    RSS0_xi[i] = regression(psi_xi, sample.T[i])[3]
    R_xi[i] = (RSS0_xi[i] - RSS_xi[i])/RSS0_xi[i]
    
    
print("R_xi^2 =", np.around((R_xi**2), decimals=3), "-> мультиколлинеарности нет\n")

R_xi^2 = [0.022 0.013 0.003 0.009 0.009] -> мультиколлинеарности нет



In [14]:
tmp = np.copy(sample)
psi = np.empty((N, 6))

for i in range(N):
     psi[i] = np.insert(tmp[i], 0, 1)
        
b, b_p_value, RSSy, RSS0y, R = regression(psi, eta)

F = psi.T @ psi
F_inv = np.linalg.inv(F)
e = (eta - psi @ b.T).T

print(" b =", np.around((b), decimals=3))
print(" p-value =", np.around((b_p_value), decimals=3),'-> 3 (и иногда 5) не значимы\n')

 b = [ 1.386  2.877 -2.393  0.716  1.197 -0.46 ]
 p-value = [0.    0.    0.    0.125 0.013 0.37 ] -> 3 (и иногда 5) не значимы



In [15]:
print("RSS0y =", "%.3f" % RSS0y)
print("RSSy =", "%.3f" % RSSy)
print("R =", "%.3f" % R)

delta_R = R/(1 - R)*(N - 6)/5

R_p_value = 1 - stats.f.cdf(np.abs(delta_R), 5, N - 6)
print("R p-value =", "%.3f" % R_p_value)

RSS0y = 362.548
RSSy = 145.031
R = 0.600
R p-value = 0.000


In [18]:
x0 = np.array([0, 0, 0, 0, 0])
psi_x0 = np.insert(x0, 0, 1)

eta0 = np.random.normal(C[0] + C[1:] @ x0.T, 1.5)
print("eta0 =", "%.3f" % eta0)

y0 = b[0] + b[1:] @ x0.T
print("y0 =", "%.3f" % y0)

delta = stats.t.ppf(1.95/2, N - 6)*(1 + (psi_x0 @ F_inv) @ psi_x0.T)**(1/2)*(RSSy/(N - 6))**(1/2)

print('[', "%.3f" % (y0 - delta),':', "%.3f" % (y0 + delta), "] - дов. интервал\n")

eta0 = 1.655
y0 = 1.386
[ -2.311 : 5.084 ] - дов. интервал



In [19]:
I=0
for i in range(N):
    for j in range (i + 1, N):
        if(e[i] > e[j]):
            I += 1

delta = (I - N*(N - 1)/4)/(N**3/36)**(1/2)

p_value = 2*(1 - stats.norm.cdf(abs(delta)))
print(" p-value =", "%.3f" % p_value, '-> не отвергаем гипотезу случайности ошибок\n')

 p-value = 0.471 -> не отвергаем гипотезу случайности ошибок



In [20]:
M = 5000
D = np.zeros(M)
s = np.zeros((M,N))
sigma_est = np.mean((e - np.mean(e))**2)**(1/2)

def sup(z, sigma):
    
    w = np.zeros(N)
    
    ecdf = sm.distributions.ECDF(z)
    u = ecdf(z)
    v = stats.norm.cdf(z, 0, sigma)
    w[0] = np.max((v[0], np.abs(v[0] - u[0])))
    
    for i in range(1, N):
        w[i] = np.max((np.abs(v[i] - u[i]), np.abs(v[i] - u[i - 1])))
    
    return np.max(w)

for i in range(M):
    for j in range(N):
        a = np.random.uniform()
        s[i][j] = stats.norm.ppf(a, 0, sigma_est)
    sigma = np.mean((s[i] - np.mean(s[i]))**2)**(1/2)
    D[i] = sup(s[i], sigma)
    
D = np.sort(D)     
d = sup(e, sigma_est)
p_value = 0

for i in range(M):
    if (D[i] > d):
        p_value = 1 - i/M
        break
        
print("p-value =", "%.3f" % p_value, "-> не отвергаем гипотезу норм. распр. ошибок")

p-value = 0.569 -> не отвергаем гипотезу норм. распр. ошибок


In [21]:
RSS_CV = 0

for i in range(N):    
    tmp = np.delete(sample, i, axis = 0)
    eta_CV = np.delete(eta, i, axis = 0)
    psi_CV = np.empty([N - 1, 6])
    for i in range(N - 1):
         psi_CV[i] = np.insert(tmp[i], 0, 1)

    F_CV = psi_CV.T @ psi_CV
    F_inv_CV = np.linalg.inv(F_CV)
    b_CV = (F_inv_CV @ psi_CV.T) @ eta_CV.T
    RSS_CV += (eta[i] - b_CV[0] - b_CV[1:] @ sample[i])**2

R_CV = (RSS0y - RSS_CV)/RSS0y
print("R_CV =", "%.3f" % R_CV, "-> высокая точность предсказаний")

R_CV = 0.653 -> высокая точность предсказаний


In [22]:
x = sample[0]
y = np.array([np.random.normal(C[0] + C[1:] @ x.T, 1.5) \
              for i in range(5)])

delta = (N - 6)*((y - np.mean(y)) @ (y - np.mean(y)).T)/RSSy/4
p_value = 1 - stats.f.cdf(np.abs(delta), 4, 5)

print("p-value =", "%.3f" % p_value, '-> модель регрессии адекватна')

p-value = 0.728 -> модель регрессии адекватна


In [24]:
print(np.argmax(b_p_value) - 1, 'кси наименее значим')

psi_simp = np.delete(psi, np.argmax(b_p_value), axis = 1)
b_simp, b_p_value_simp, RSSy_simp, RSS0y_simp, R_simp = regression(psi_simp, eta)
print("\n Для упрощенной модели:")
print("b =", np.around((b_simp), decimals=3))
print("p-value =", np.around((b_p_value_simp), decimals=3))

delta_R_simp = R_simp/(1 - R_simp)*(N - 5)/4
p_value_simp = 1 - stats.f.cdf(np.abs(delta_R_simp), 4, N - 5)

print("p-value =", "%.3f" % p_value_simp)

print("\nВ сравнении:")
delta_regression = (RSSy_simp - RSSy)/RSSy*(N - 6)
p_value = 1 - stats.f.cdf(np.abs(delta), 1, N - 6)
print("p-value =", "%.3f" % p_value, "-> упрощение не делает погоды:)")

4 кси наименее значим

 Для упрощенной модели:
b = [ 1.415  2.813 -2.357  0.746  1.08 ]
p-value = [0.    0.    0.    0.109 0.019]
p-value = 0.000

В сравнении:
p-value = 0.475 -> упрощение не делает погоды:)


In [25]:
M = 1000
B_delta = np.empty(M)

for i in range(M): 
    psi1 = np.empty((N,6))
    eta1 = np.empty(N)
    for j in range(N):
        q = np.random.choice(range(N))
        psi1[j] = psi[q]
        eta1[j] = eta[q]
        
    b1, b1_p_v, RSSy1, RSS0y1, Ry1 = regression(psi1, eta1)
    b2, b2_p_v, RSSy2, RSS0y2, Ry2 = \
    regression(np.delete(psi1, np.argmax(b1_p_v), axis = 1), eta1)
    
    B_delta[i] = (RSSy2/RSSy1 - 1)*(N - 6)
    
B_delta = np.sort(B_delta)
p_value = 0

for i in range(M):
    if (B_delta[i] > delta_regression):
        p_value = 1 - i/M
        break
        
print("p-value =", "%.3f" % p_value)

p-value = 0.342
